# 한국어 욕설을 인공신경망을 통해 분류합니다.

이 노트북은 2runo님의 한국어 욕설 데이터를 가지고 텍스트를 욕설 또는 욕설이 아님으로 분류합니다.
이 인공신경망은 이진(binary) 또는 클래스(Class)가 두개인 분류 모델입니다.

## 데이터를 전처리하고 인공신경망을 훈련하기 위해 모델들을 임포트합니다.

In [5]:
# tensorflow와 tf.kears를 임포트합니다.
import tensorflow as tf
from tensorflow import keras
print(f"텐서플로우 버전 : {tf.__version__}")

# Scikit-learn 라이브러리를 임포트합니다.
from sklearn.model_selection import train_test_split

# 헬퍼(helper)라이브러리를 임포트합니다.
import numpy as np
import pandas as pd
import matplotlib as plt

# 시작하기전에 라이브러리들의 버젼들을 출력합니다.
print(f"텐서플로우 버전 : {tf.__version__}")
print(f"넘파이 버전 : {np.__version__}")
print(f"판다스 버전 : {pd.__version__}")
print(f"메트풀룻 버전 : {plt.__version__}")

텐서플로우 버전 : 2.5.0
텐서플로우 버전 : 2.5.0
넘파이 버전 : 1.19.5
판다스 버전 : 1.3.0
메트풀룻 버전 : 3.4.2


## 학습 데이터를 사전 처리합니다.


### 데이터 불러오기

우리가 사용할 데이터 세트는 앞서 말씀드렸듯이 2urno님의
한국어 욕설 리스트 데이터입니다. 원본파일은 .txt 확장자로 되어있었으나, 편의상
사전적으로 .csv 파일으로 변환하였습니다.

이 .csv 파일은 문장이 적혀있는 sentence 열, 그리고 욕설인지 아닌지 분류하는 타깃이 적혀있는
target 열이 있습니다. target열은 만약 해당 문장이 욕설일경우는 양성 클래스인 1, 욕설이 아닌 경우
음성 클래스인 0이 할당되어 있는 구조입니다.

학습을 위해 데이터를 가져오는 함수를 제작한뒤 같은열에서 실행하겠습니다.

In [10]:
from typing import Any

def load_data() -> Any:
    swear_df = pd.read_csv('./dataset/2runos-swear.csv')
    sentence = swear_df['sentence']
    target = swear_df['target']
    return np.array(sentence) ,np.array(target)

total_sentence, total_target = load_data()
print(total_sentence)

['좌배 까는건 ㅇㅂ' '집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ'
 ' 개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아' ... ' 너가 한 말 중에' '제갈대중 ㅇㅂ'
 '우리나라교회는 악마들이모여 주뎅이 처벌리고 ']


### 데이터 사전 처리(전처리)하기


#### 문장들을 정수화하기

일반적으로, 인공신경망에 데이터를 전달할때는 정수형 데이터를 전달합니다.
그리하여 문장을 정수로 만들수 있는 함수를 제작하겠습니다.


In [27]:
def split_the_list_original(input_ls : list) -> list:
    two_dimensional_paramls = []
    for input_string in input_ls:
        appendls = list(str(input_string).split())
        two_dimensional_paramls.append(appendls)
    return two_dimensional_paramls

def making_sentence_dict():
    original_df = pd.read_csv('./dataset/sentence.csv')
    sentence_dict = {}
    dict_number = 0
    for element in original_df['sentence']:
        dict_number += 1
        sentence_dict[f"{element}"] = dict_number
    return sentence_dict


def change_the_sentence_to_int(sentence_list):
    sentence_dict = making_sentence_dict()
    outputls = []
    sentence_list = split_the_list_original(sentence_list)
    for sentence in sentence_list:
        stls = []
        for words in sentence:
            try:
                word = sentence_dict[words]
                stls.append(word)
            except:
                stls.append(0) # unknown
        outputls.append(stls)
    return outputls

In [28]:
total_int_sentence = change_the_sentence_to_int(total_sentence)
print(total_int_sentence)

[[618, 25497, 18847], [4527, 14787, 9007, 19522, 6293, 9266, 21426, 6189, 4914], [2290, 4587, 18575, 24031, 4545, 14308, 7195, 10075, 11553], [21707, 9343, 5512], [24511, 25652, 13515, 1531], [1534, 18426, 23638, 15484, 2589, 15202, 19447, 19102, 20455, 25199, 19324, 6862, 14041, 400, 20565, 24197, 20353, 8314, 9162, 7162, 9565, 10520, 14386], [12135], [21190, 11911, 27262, 7071, 1064, 24949, 6500, 20298], [21069, 27128, 12666, 26325, 2579, 19340, 19685, 22437], [4702, 11117, 912, 17778, 1524, 23522, 11838, 14315, 25373], [5620], [19517], [1560, 535, 16691, 15517, 20922, 2272, 20915], [20152, 3812, 11599, 22232, 17700, 3140, 22083, 19073, 14315, 12100, 7028, 19746, 11401], [7591, 10476, 1272, 14679, 14407], [4784, 9292, 16263, 26002, 24156, 24830, 4932], [9214, 13664, 15506, 7755, 11361, 3524, 7736, 2249], [18819, 17656, 7184, 27682], [15215, 2113, 9595, 3549, 8621, 6578, 9544, 22182, 4540], [14490, 27115, 11830, 19064, 3790, 12403, 5237, 23589, 11824, 18433, 5514, 12772, 24265, 7453, 

#### 훈련세트와 테스트세트 분리

In [30]:
print(f"훈련 셈플 : {len(total_int_sentence)}, 레이블 : {len(total_target)}")

훈련 셈플 : 5825, 레이블 : 5825


In [31]:
train_data, test_data, train_labels, test_labels = train_test_split(total_int_sentence, total_target, random_state=42)